# Getting data from Spotify API

https://developer.spotify.com/web-api/

## Import things

In [13]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import numpy as np

## Creat API client with the right keys

In [ ]:
client_credentials_manager = SpotifyClientCredentials('0bc389e00d214082bf48acca68452036', '9b9ad7bee99b490cbc657a388de74dd0')
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Load the data

In [14]:
songs_lyrics = pd.read_csv('./songs_lyrics.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
songs_lyrics.head(3)

,Unnamed: 0,Unnamed: 0_x,Position,Track Name,Artist,Streams,URL,Date,Year,Month,Day,Country,Region,art_song,Unnamed: 0_y,Band,Lyrics,Song,_merge
0,0,2333254,197,Bailemos juntos,#TocoParaVos,1005,https://open.spotify.com/track/4LEzZn6vpMkQAQu...,2017-01-01,2017,1,1,uy,sa,#TocoParaVos - Bailemos juntos,NaN,NaN,NaN,NaN,left_only
1,1,149761,162,Enganchado #TocoParaVos - Cumbia Mix,#TocoParaVos,14864,https://open.spotify.com/track/7zW4IqrzU6dTLw3...,2017-01-07,2017,1,7,ar,sa,#TocoParaVos - Enganchado #TocoParaVos - Cumbi...,NaN,NaN,NaN,NaN,left_only
2,2,149989,190,Enganchado #TocoParaVos - Cumbia Mix,#TocoParaVos,11133,https://open.spotify.com/track/7zW4IqrzU6dTLw3...,2017-01-08,2017,1,8,ar,sa,#TocoParaVos - Enganchado #TocoParaVos - Cumbi...,NaN,NaN,NaN,NaN,left_only


### Get all unique song uri's

In [17]:
uris = songs_lyrics['URL'].str.replace('https://open.spotify.com/track/', 'spotify:track:')

In [18]:
uris.shape

(3355829,)

In [20]:
uris2 = uris.drop_duplicates()

In [21]:
uris2.shape

(21372,)

### Make a list of lists of uris (50 each)

https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks

In [23]:
l = uris2.values
n = 50

lists = [l[i:i + n] for i in range(0, len(l), n)]

### Use Spotify API to get data for songs

In [25]:
def get_data(lst):
    results = spotify.audio_features(tracks=lst)
    return results

In [29]:
all_results = []

In [30]:
for lst in lists:
    r = get_data(lst)
    all_results = all_results + r

In [42]:
len(all_results)

21372

In [39]:
# Remove non values (3) and make result to a list
all_avail_results = list(filter(None, all_results))

In [43]:
len(all_avail_results)

21369

## Save result as data frame

In [40]:
df = pd.DataFrame(all_avail_results)

In [44]:
df.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.1160,https://api.spotify.com/v1/audio-analysis/4LEz...,0.777,203000,0.596,4LEzZn6vpMkQAQuwFInnSh,0.000,0,0.2750,-7.549,0,0.0523,111.938,4,https://api.spotify.com/v1/tracks/4LEzZn6vpMkQ...,audio_features,spotify:track:4LEzZn6vpMkQAQuwFInnSh,0.579
1,0.0197,https://api.spotify.com/v1/audio-analysis/7zW4...,0.658,180036,0.435,7zW4IqrzU6dTLw3CM72Qbo,0.000,11,0.1150,-12.955,0,0.0492,108.994,4,https://api.spotify.com/v1/tracks/7zW4IqrzU6dT...,audio_features,spotify:track:7zW4IqrzU6dTLw3CM72Qbo,0.736
2,0.0124,https://api.spotify.com/v1/audio-analysis/1A3y...,0.837,149160,0.605,1A3yLIqelu1D7MCtw06Cu7,0.000,0,0.0715,-6.122,0,0.0798,107.998,4,https://api.spotify.com/v1/tracks/1A3yLIqelu1D...,audio_features,spotify:track:1A3yLIqelu1D7MCtw06Cu7,0.723
3,0.0155,https://api.spotify.com/v1/audio-analysis/6hTv...,0.812,183004,0.723,6hTvQmZwsvcRpHQbLUwpZE,0.029,6,0.0932,-5.247,0,0.0390,96.948,4,https://api.spotify.com/v1/tracks/6hTvQmZwsvcR...,audio_features,spotify:track:6hTvQmZwsvcRpHQbLUwpZE,0.636
4,0.0281,https://api.spotify.com/v1/audio-analysis/47K5...,0.691,164587,0.807,47K5fP6VdTpxF4Nxk0chwQ,0.000,2,0.0844,-3.786,1,0.0818,107.053,4,https://api.spotify.com/v1/tracks/47K5fP6VdTpx...,audio_features,spotify:track:47K5fP6VdTpxF4Nxk0chwQ,0.771


Explanation for the Spotify API columns<BR />
https://developer.spotify.com/web-api/get-audio-features/

## Save the data as a CSV-file

In [45]:
# save data to csv-file
df.to_csv('spotify_api_data.csv')